In [1]:
from pathlib import Path
import random
from random import randrange
import re
from typing import Any, Tuple


from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pytube import YouTube
from pytube.cli import on_progress

from utils import settings
from utils.CONSTANTS import background_options
from utils.console import print_step, print_substep


In [29]:
def get_start_and_end_times(video_length: int, length_of_clip: int) -> Tuple[int, int]:
    """Generates a random interval of time to be used as the background of the video.

    Args:
        video_length (int): Length of the video
        length_of_clip (int): Length of the video to be used as the background

    Returns:
        tuple[int,int]: Start and end time of the randomized interval
    """
    random_time = randrange(180, int(length_of_clip) - int(video_length))
    return random_time, random_time + video_length

In [28]:
def get_background_config():
    """Fetch the background/s configuration"""
    try:
        choice = str(settings.config["settings"]["background"]["background_choice"]).casefold()
    except AttributeError:
        print_substep("No background selected. Picking random background'")
        choice = None

    # Handle default / not supported background using default option.
    # Default : pick random from supported background.
    if not choice or choice not in background_options:
        choice = random.choice(list(background_options.keys()))

    return background_options[choice]

In [24]:
def chop_background_video(background_config: Tuple[str, str, str, Any], video_length: int):
    """Generates the background footage to be used in the video and writes it to assets/temp/background.mp4

    Args:
        background_config (Tuple[str, str, str, Any]) : Current background configuration
        video_length (int): Length of the clip where the background footage is to be taken out of
    """

    print_step("Finding a spot in the backgrounds video to chop...✂️")
    choice = f"{background_config[2]}-{background_config[1]}"
    background = VideoFileClip(f"assets/backgrounds/{choice}")

    start_time, end_time = get_start_and_end_times(video_length, background.duration)
    try:
        ffmpeg_extract_subclip(
            f"assets/backgrounds/{choice}",
            start_time,
            end_time,
            targetname=f"assets/backgrounds/cut_vid/background.mp4",
        )
    except (OSError, IOError):  # ffmpeg issue see #348
        print_substep("FFMPEG issue. Trying again...")
        with VideoFileClip(f"assets/backgrounds/{choice}") as video:
            new = video.subclip(start_time, end_time)
            new.write_videofile(f"assets/backgrounds/cut_vid/background.mp4")
    print_substep("Background video chopped successfully!", style="bold green")
    return background_config[2]

In [2]:
def download_background(background_config: Tuple[str, str, str, Any]):
    """Downloads the background/s video from YouTube."""
    Path("./assets/backgrounds/").mkdir(parents=True, exist_ok=True)
    # note: make sure the file name doesn't include an - in it
    uri, filename, credit, _ = background_config
    if Path(f"assets/backgrounds/{credit}-{filename}").is_file():
        return
    print_step(
        "We need to download the backgrounds videos. they are fairly large but it's only done once. 😎"
    )
    print_substep("Downloading the backgrounds videos... please be patient 🙏 ")
    print_substep(f"Downloading {filename} from {uri}")
    YouTube(uri, on_progress_callback=on_progress).streams.filter(res="1080p").first().download(
        "assets/backgrounds", filename=f"{credit}-{filename}"
    )
    print_substep("Background video downloaded successfully! 🎉", style="bold green")

In [13]:
# choice = random.choice(list(background_options.keys()))
choice = "minecraft"
bg_config = background_options[choice]
download_background(bg_config)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ We need to download the backgrounds videos. they are fairly large but it's only done once. 😎                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Downloading the backgrounds videos... please be patient 🙏

Downloading parkour.mp4 from https://www.youtube.com/watch?v=n_Dv4JMiwK8

Background video downloaded successfully! 🎉

In [27]:
length = 20
chop_background_video(bg_config, length)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Finding a spot in the backgrounds video to chop...✂️                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                                            
                                                              
t:   1%|          | 2/300 [04:57<00:31,  9.53it/s, now=None]  

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


Background video chopped successfully!

'bbswitzer'

In [32]:
#!/usr/bin/env python3
import multiprocessing
import os
import re
from os.path import exists
from typing import Tuple, Any
from moviepy.audio.AudioClip import concatenate_audioclips, CompositeAudioClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.video.VideoClip import ImageClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
from moviepy.video.compositing.concatenate import concatenate_videoclips
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from rich.console import Console

from utils.cleanup import cleanup
from utils.console import print_step, print_substep
from utils.video import Video
from utils.videos import save_data
from utils import settings

In [ ]:
console = Console()
W, H = 1080, 1920

def name_normalize(name: str) -> str:
    name = re.sub(r'[?\\"%*:|<>]', "", name)
    name = re.sub(r"( [w,W]\s?\/\s?[o,O,0])", r" without", name)
    name = re.sub(r"( [w,W]\s?\/)", r" with", name)
    name = re.sub(r"(\d+)\s?\/\s?(\d+)", r"\1 of \2", name)
    name = re.sub(r"(\w+)\s?\/\s?(\w+)", r"\1 or \2", name)
    name = re.sub(r"\/", r"", name)
    name[:30]
    return name


def make_final_video(
    number_of_clips: int,
    length: int,
    background_config: Tuple[str, str, str, Any],
):
    """Gathers audio clips, gathers all screenshots, stitches them together and saves the final video to assets/temp
    Args:
        number_of_clips (int): Index to end at when going through the screenshots'
        length (int): Length of the video
        background_config (Tuple[str, str, str, Any]): The background config to use.
    """

    print_step("Creating the final video 🎥")
    VideoFileClip.reW = lambda clip: clip.resize(width=W)
    VideoFileClip.reH = lambda clip: clip.resize(width=H)
    times_to_run = 1
    opacity = 0.9
    transition = 0.2 
    background_clip = (
        VideoFileClip(f"assets/backgrounds/cut_vid/background.mp4")
        .without_audio()
        .resize(height=H)
        .crop(x1=1166.6, y1=0, x2=2246.6, y2=1920)
    )

    # Gather all audio clips
    audio_clips = [AudioFileClip(f"assets/mp3/{i}.mp3") for i in range(number_of_clips)]
    audio_clips.insert(0, AudioFileClip(f"assets/mp3/title.mp3"))
    audio_concat = concatenate_audioclips(audio_clips)
    audio_composite = CompositeAudioClip([audio_concat])

    console.log(f"[bold green] Video Will Be: {length} Seconds Long")
    # add title to video
    image_clips = []
    # Gather all images
    new_opacity = 1 if opacity is None or float(opacity) >= 1 else float(opacity)
    new_transition = 0 if transition is None or float(transition) > 2 else float(transition)
    image_clips.insert(
        0,
        ImageClip(f"assets/png/title.png")
        .set_duration(audio_clips[0].duration)
        .resize(width=W - 100)
        .set_opacity(new_opacity)
        .crossfadein(new_transition)
        .crossfadeout(new_transition),
    )

    for i in range(0, number_of_clips):
        image_clips.append(
            ImageClip(f"assets/png/part_{i}.png")
            .set_duration(audio_clips[i + 1].duration)
            .resize(width=W - 100)
            .set_opacity(new_opacity)
            .crossfadein(new_transition)
            .crossfadeout(new_transition)
        )

    img_clip_pos = background_config[3]
    image_concat = concatenate_videoclips(image_clips).set_position(
        img_clip_pos
    )  # note transition kwarg for delay in imgs
    image_concat.audio = audio_composite
    final = CompositeVideoClip([background_clip, image_concat])

    filename = "a.mp4"

    final.write_videofile(
        f"assets/temp.mp4",
        fps=30,
        audio_codec="aac",
        audio_bitrate="192k",
        verbose=False,
        threads=multiprocessing.cpu_count(),
    )
    ffmpeg_extract_subclip(
        f"assets/temp.mp4",
        0,
        length,
        targetname=f"results/{filename}",
    )
    print_step("Removing temporary files 🗑")
    cleanups = cleanup(id)
    print_substep(f"Removed {cleanups} temporary files 🗑")
    print_substep("See result in the results folder!")
